# RAG using Function calling for tabular data ingestion

This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models.

`tools` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

Within the `tools` parameter, if the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can be forced to use a specific function by setting the `tool_choice` parameter to `{"name": "<insert-function-name>"}`. The API can also be forced to not use any function by setting the `tool_choice` parameter to `"none"`. If a function is used, the output will contain `"finish_reason": "function_call"` in the response, as well as a `tool_choice` object that has the name of the function and the generated function arguments.

### Overview

This notebook contains the following 2 sections:

- **How to generate function arguments:** Specify a set of functions and use the API to generate function arguments.
- **How to call functions with model generated arguments:** Close the loop by actually executing functions with model generated arguments.

## Function calling using the tools feature of the OpenAI API

In [262]:
import json
import openai
import requests

GPT_MODEL = "gpt-3.5-turbo-0613"

In [263]:
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')



Define a utility function for making calls to the Chat Completions API 

In [264]:
# This is the function that is used to call the Open AI Chat completion API
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if tools is not None:
        json_data.update({"tools": tools})
    if tool_choice is not None:
        json_data.update({"tool_choice": tool_choice})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


### Defining a function spec

Let's create a function specifications to get sales information for chocolates. We'll pass this function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [266]:


tools = [ 
          {
            "type": "function",
           "function":
            { 
              "name": "get_sales_info", 
              "description": "Get the sales information for chocolates",
              "parameters": 
              {
                  "type": "object",
                  "properties": 
                  {
                    
                  }
                
              }
            }
          }
        ]


In [267]:
planogram_prompt = '''You are an expert planogram designer. I need your help in designing a planogram for a shelf, in a Retail Store in Bangalore.  The shelf is in the chocolates aisle. The shelf has 5 columns and 4 rows. Each cell in the shelf can hold one chocolate box. So the shelf can hold a total of 5 x 4 = 20 chocolate boxes. I want to prioritize in the following order(highest priority is listed first and lowest is listed last):market share, then margin share, then new product, then core product. It is very important that the returned planogram should be in the format given below. Do not leave any cell empty. Can you help me with the planogram design ?

[["AAA", "AAA", "AAA", "AAA", "AAA"],
 ["AAA", "AAA", "AAA", "AAA", "AAA"],
 ["AAA", "AAA", "AAA", "AAA", "AAA"],
 ["AAA", "AAA", "AAA", "AAA", "AAA"]]'''

In [268]:
def get_sales_info():
    with open('sales_info.json') as fp:
        sales_info = json.load(fp)
    return sales_info


#### Forcing the use of specific functions or no function

We can force the model to use a specific function, if the function named "get_sales_info", by using the function argument. 

In [269]:
# in this cell we force the model to use the function named "get_sales_info"
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": planogram_prompt})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_sales_info"}}
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_RSjhpuBhzb5w6VgjlfN1l4Tf',
   'type': 'function',
   'function': {'name': 'get_sales_info', 'arguments': '{}'}}]}

In [270]:
# This function will check the intermediate json response from the model to see if is has asked us to "call" a function. 
# If there is a function call request with the correct function name, that function will be invoked

def execute_function_call(message):
    if message["tool_calls"][0]["function"]["name"] == "get_sales_info":
        results = get_sales_info()
    else:
        results = f"Error: function {message['tool_calls'][0]['function']['name']} does not exist"
    return results

In [271]:


assistant_message = chat_response.json()["choices"][0]["message"]
# chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
if assistant_message.get("tool_calls"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "tool", "tool_call_id": assistant_message["tool_calls"][0]['id'], "name": assistant_message["tool_calls"][0]["function"]["name"], "content": f"{results}"})

print(f"Assistant Message = {assistant_message}")
print(f"Messages = {messages}")
chat_response_2 = chat_completion_request(messages, tools)
chat_response_2.json()


Assistant Message = {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_RSjhpuBhzb5w6VgjlfN1l4Tf', 'type': 'function', 'function': {'name': 'get_sales_info', 'arguments': '{}'}}]}
Messages = [{'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}, {'role': 'user', 'content': 'You are an expert planogram designer. I need your help in designing a planogram for a shelf, in a Retail Store in Bangalore.  The shelf is in the chocolates aisle. The shelf has 5 columns and 4 rows. Each cell in the shelf can hold one chocolate box. So the shelf can hold a total of 5 x 4 = 20 chocolate boxes. I want to prioritize in the following order(highest priority is listed first and lowest is listed last):market share, then margin share, then new product, then core product. It is very important that the returned planogram should be in the format given below. Do not leave any cell empty. Can you he

{'id': 'chatcmpl-8LdFlV0SUzhNamKfd25eBRooy8IP1',
 'object': 'chat.completion',
 'created': 1700166469,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Based on the sales information provided, here is the planogram for the shelf in the chocolates aisle:\n\n```\n[["Munch", "Munch", "Milky bar", "KitKat", "AAA"],\n ["Milky bar", "KitKat", "AAA", "AAA", "AAA"],\n ["AAA", "AAA", "AAA", "AAA", "AAA"],\n ["AAA", "AAA", "AAA", "BarOne", "BarOne"]]\n```\n\nExplanation:\n- The highest priority is given to market share. Therefore, the chocolates with the highest market share percentages are placed first.\n- The second priority is margin share, followed by new product and core product.\n- AAA is used to fill the remaining cells on the shelf to ensure that all cells are filled.\n\nPlease note that the planogram is based on the available sales information and the specified prioritization order.'},
   'finish_reason': 'stop'}],
 'usage': {